<a href="https://colab.research.google.com/github/rahulpandeycs/CS5100-COVID19-AI-PROJECT/blob/master/CS5100_COVID19_AI_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, I/O for CSV files
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import time
import datetime as dt
from datetime import date
import warnings

View the list of input data

In [13]:
for dirname, _, filenames in os.walk('data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
warnings.filterwarnings('ignore')

data/time_series_covid_19_deaths.csv
data/covid_19_data.csv
data/time_series_covid_19_confirmed.csv
data/time_series_covid_19_recovered.csv
data/COVID19_open_line_list.csv
data/COVID19_line_list_data.csv


In [36]:
Y2019_dataset_path = 'data/covid_19_data.csv'
Y2019_data = pd.read_csv(Y2019_dataset_path)
# Y2019_data['Last Update'] = Y2019_data['Last Update'].apply(pd.datetime)
Y2019_data['Last Update'] = pd.to_datetime(Y2019_data['Last Update'], format='%m/%d/%Y %H:%M')
Y2019_data.drop(['SNo'], axis = 1, inplace = True)
Y2019_data.tail()

ValueError: ignored

In [24]:
listOfCountries = Y2019_data['Country/Region'].unique().tolist()
print("\n Number of countries affected by COVID-19 virus: ", len(listOfCountries))
print(listOfCountries)


 Number of countries affected by COVID-19 virus:  212
['Mainland China', 'Hong Kong', 'Macau', 'Taiwan', 'US', 'Japan', 'Thailand', 'South Korea', 'Singapore', 'Philippines', 'Malaysia', 'Vietnam', 'Australia', 'Mexico', 'Brazil', 'Colombia', 'France', 'Nepal', 'Canada', 'Cambodia', 'Sri Lanka', 'Ivory Coast', 'Germany', 'Finland', 'United Arab Emirates', 'India', 'Italy', 'UK', 'Russia', 'Sweden', 'Spain', 'Belgium', 'Others', 'Egypt', 'Iran', 'Israel', 'Lebanon', 'Iraq', 'Oman', 'Afghanistan', 'Bahrain', 'Kuwait', 'Austria', 'Algeria', 'Croatia', 'Switzerland', 'Pakistan', 'Georgia', 'Greece', 'North Macedonia', 'Norway', 'Romania', 'Denmark', 'Estonia', 'Netherlands', 'San Marino', ' Azerbaijan', 'Belarus', 'Iceland', 'Lithuania', 'New Zealand', 'Nigeria', 'North Ireland', 'Ireland', 'Luxembourg', 'Monaco', 'Qatar', 'Ecuador', 'Azerbaijan', 'Czech Republic', 'Armenia', 'Dominican Republic', 'Indonesia', 'Portugal', 'Andorra', 'Latvia', 'Morocco', 'Saudi Arabia', 'Senegal', 'Argenti

In [25]:
Y2019_China = Y2019_data.loc[Y2019_data['Country/Region'] == 'Mainland China']
Y2019_data.tail()

,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
9730,03/28/2020,Wyoming,US,2020-03-28 23:11:06,82.0,0.0,0.0
9731,03/28/2020,Xinjiang,Mainland China,2020-03-28 23:11:06,76.0,3.0,73.0
9732,03/28/2020,Yukon,Canada,2020-03-28 23:11:06,4.0,0.0,0.0
9733,03/28/2020,Yunnan,Mainland China,2020-03-28 23:11:06,180.0,2.0,172.0
9734,03/28/2020,Zhejiang,Mainland China,2020-03-28 23:11:06,1251.0,1.0,1225.0


In [43]:
Y2019_data_new = Y2019_data[Y2019_data['Last Update'] > '2/15/2020 12:00']
Y2019_data.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


In [45]:
dataFrame = pd.DataFrame(Y2019_data_new)
dataFrame = dataFrame.loc[dataFrame['Country/Region'] == 'Italy']
dataFrame.tail()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
8285,8286,03/24/2020,NaN,Italy,2020-03-24 23:41:50,69176.0,6820.0,8326.0
8586,8587,03/25/2020,NaN,Italy,2020-03-25 23:37:49,74386.0,7503.0,9362.0
8888,8889,03/26/2020,NaN,Italy,2020-03-26 23:53:24,80589.0,8215.0,10361.0
9195,9196,03/27/2020,NaN,Italy,2020-03-27 23:27:48,86498.0,9134.0,10950.0
9502,9503,03/28/2020,NaN,Italy,2020-03-28 23:11:06,92472.0,10023.0,12384.0


In [0]:
newConfirmed = [0]
newDeaths = [0]
newRecovered = [0]
for data in range(1, (dataFrame.Confirmed).size): 
    newConfirmed.append(dataFrame.Confirmed.tolist()[data]-dataFrame.Confirmed.tolist()[data-1])
    newDeaths.append(dataFrame.Deaths.tolist()[data]-dataFrame.Deaths.tolist()[data-1])
    newRecovered.append(dataFrame.Recovered.tolist()[data]-dataFrame.Recovered.tolist()[data-1])


Using TensorFlow Backend for prediction of deaths and number of confirmed cases

In [0]:
from numpy import array
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import LSTM, Dropout, Dense

In [0]:
def format_sequence(sequence, n_steps):
    X, Y = list(), list()
    for data in range(len(sequence)):
      end_datax = data + n_steps #End of this pattern
      if end_datax > len(sequence) - 1:
        break  # break if we went beyond the sequence
      seqX, seqY = sequence[data: end_datax], sequence[end_datax]
      X.append(seqX)
      Y.append(seqY)
    return array(X), array(Y)    

In [0]:
unformattedSequence = np.array(newConfirmed)
nSteps = 5
X, Y = format_sequence(unformattedSequence, nSteps)


nFeatures = 1
X = X.reshape((X.shape[0], X.shape[1], nFeatures))

Define model and adding neural network layers

In [66]:
LSTMModel = Sequential()
LSTMModel.add(LSTM(50, activation='relu', return_sequences=True, input_shape = (nSteps, nFeatures)))
LSTMModel.add(Dropout(0.1))
LSTMModel.add(LSTM(50, activation='relu', return_sequences=True))
LSTMModel.add(Dropout(0.1))
LSTMModel.add(LSTM(50, activation='relu'))
LSTMModel.add(Dense(1))

#Added two LSTM layers

start = time.time()
LSTMModel.compile(optimizer='adam', loss='mse')
print('Compilation time :', time.time() - start)
print('\n')
LSTMModel.summary()

Compilation time : 0.01148223876953125


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 5, 50)             10400     
_________________________________________________________________
dropout (Dropout)            (None, 5, 50)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 5, 50)             20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 50)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 50,851
Trainable params: 50,851
Non-trainable params: 0
___________

In [68]:
LSTMModel.fit(X,Y, epochs=300, verbose= 0)
xInput = np.array(newConfirmed)[-nSteps:]
xInput = xInput.reshape((1, nSteps, nFeatures))
yHat =LSTMModel.predict(xInput, verbose=0)

forecastedCases = int(round(yHat[0][0]))
print('Case forecast for today: {}'.format(forecastedCases))

accuracy = 100*(1-np.abs(forecastedCases - newConfirmed[-1])/newConfirmed[-1])
print('Forecasted accuracy: {:.2f} %'.format(accuracy))

Case forecast for today: 5184
Forecasted accuracy: 86.78 %


Tomorrow forecast

In [70]:
print('and for tomorrow?')
x_input2 = np.array(newConfirmed)[(-nSteps+1):]
x_input2 = np.concatenate((x_input2, yHat), axis=None)
x_input2 = x_input2.reshape((1, nSteps, nFeatures))
yhat2 = LSTMModel.predict(x_input2, verbose=0)
print('tomorrow there might be {:.0f} cases'.format(int(yhat2[0][0])))

and for tomorrow?
tomorrow there might be 6159 cases
